In [1]:
import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
import pydotplus #Graphviz要用到的库

In [8]:
#sl: satisfaction_level —— False: MinMinMaxScaler ; True: StanStandardScaler 
#这里用False和True来表示MinMinMaxScaler还是StanStandardScaler 
#其他同理
#dp:department——False:LabelEncoder; True: OneHotEncoder
#salary也是
#lower_d=False表示不降维 ld_n=1表示降维到1维
def hr_preprocessing(sl = False,le = False,npr=False,amh=False,tsc=False,
                     wa=False,pl5=False,dp=False,slr=False,lower_d=False,ld_n=1):
    df = pd.read_csv('./HR.csv')
    #1、清洗数据
    df = df.dropna(subset = ['satisfaction_level','last_evaluation'])
    df = df[df['satisfaction_level']<=1][df['salary']!='nme']
    #2、得到标注
    label = df['left']
    df = df.drop('left',axis = 1)
    #3、特征选择
    #（因为本文的特征本来就不多，暂时不做）
    #4、特征处理
        #这里对satisfaction_level等用了归一化 
        #对department和salary用数值化
    scaler_lst = [sl,le,npr,amh,tsc,wa,pl5]#这是一个参数
    column_lst = ['satisfaction_level','last_evaluation', "number_project",
                  "average_monthly_hours", "time_spend_company", "Work_accident",
                  "promotion_last_5years"]
    for i in range(len(scaler_lst)):#遍历scaler_lst
        if not scaler_lst[i]:
            df[column_lst[i]] = \
            MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
            #这里的\只是要换行的意思
        else:
            df[column_lst[i]] = \
            StandardScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
    scaler_lst = [dp,slr]#这是一个参数
    column_lst = ['department','salary']
    for i in range(len(scaler_lst)):#遍历scaler_lst
        if not scaler_lst[i]:#LabelEncoder标签化
            if column_lst[i] == 'salary':
                df[column_lst[i]] = [map_salary(s) for s in df['salary'].values]
            else:
                df[column_lst[i]] = LabelEncoder().fit_transform(df[column_lst[i]])
            df[column_lst[i]] = \
            MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
        else:#OneHotEncoder独热
            df = pd.get_dummies(df,columns = [column_lst[i]])
            #直接 LabelEncoder()会自动进行升序的处理 所以需要下面的01处理
    #考虑是否降维
    if lower_d:
        return PCA(n_components=ld_n).fit_transform(df.values),label
    return df,label#这里也同时返回来label

d = dict([('low',0),('medium',1),('high',2)])#01
def map_salary(s):
    return d.get(s,0)#这里给low medium high定义数字 如果没有找到 则为0

#开始建模
#sklearn中没有可以一次性将数据划分成验证集、训练集、测试集的包，所以需要进行两步划分
#features是特征的意思
#这里要切分为6:2:2

#分类
def hr_modeling(features,label):
    from sklearn.model_selection import train_test_split#这是直接切分训练集和测试集的函数
    f_v = features.values
    f_names = features.columns.values#插入特征名称
    l_v = label.values
    X_tt,X_validation,Y_tt,Y_validation = train_test_split(f_v,l_v,test_size = 0.2)
    #这里表示验证集占多少比例 这里验证集割离出去了
    #X_tt是训练集部分 X_validation是验证集部分 Y_tt,Y_validation是它的标注
    X_train,X_test,Y_train,Y_test = train_test_split(X_tt,Y_tt,test_size = 0.25)
    #X_train是训练集 X_test是测试集
    #print(len(X_train),len(X_validation),len(X_test))#这里是打印出各个集合的数量
    
    
    from sklearn.neighbors import NearestNeighbors,KNeighborsClassifier#KNN(分类器)
    from sklearn.metrics import accuracy_score,recall_score,f1_score#KNN(分类器)
    from sklearn.naive_bayes import GaussianNB,BernoulliNB#贝叶斯的库 有：一个高斯朴素贝叶斯 一个伯努利朴素贝叶斯
    #在用朴素贝叶斯时 数据必须是离散的 而离散值是二值的话（就是0和1组成） 用伯努利朴素贝叶斯 
    #如果数据是连续的 在伯努利朴素贝叶斯算法下 也会先二值化
    #高斯朴素贝叶斯就是先假设特征是高斯分布的
    from sklearn.tree import DecisionTreeClassifier,export_graphviz#决策树和Graphviz Graphviz同于输出决策树
    from sklearn.externals.six import StringIO#Graphviz可能要用到
    from sklearn.svm import SVC
    from sklearn.ensemble import RandomForestClassifier#随机森林 默认Gini
    from sklearn.ensemble import AdaBoostClassifier#AdaBoost
    from sklearn.linear_model import LogisticRegression#逻辑斯特回归
    #n_neighbors: 近邻数量 默认为5 
    #leaf_size :最大叶子数，会影响性能与内存 
    #n_jobs : 并行参数
    
    from sklearn.ensemble import GradientBoostingClassifier#提升树
    
    #人工神经网络
    from keras.models import Sequential#人工神经网络 Sequential模型 是人工神经网络的容器 序列化的 层级化的
    from keras.layers.core import Dense,Activation#Dense是神经网络的稠密层  Activation是激活函数
    from keras.optimizers import SGD#随机梯度下降算法
    mdl = Sequential()#建立神经网络模型
    mdl.add(Dense(50,input_dim = len(f_v[0])))#Dense(50)表示它的输入层 50是下个隐含层的神经元的个数 
    #input_dim指输入的维度 和数据本身密切相关 len(f_v[0])这里我们选择了第一条数据
    mdl.add(Activation('sigmoid'))#sigmoid是一个激活函数 因为这里数据量不是很大 所有用这个函数
    mdl.add(Dense(2))#这里是添加输出层 设置输出几维 因为这里我们的标注只有两个值 所有对应的是2
    mdl.add(Activation('softmax'))#这里保证它归一化
    sgd = SGD(lr = 0.1)#这里添加一个优化器 lr指它的学习率 就是梯度下降算法的阿尔法
    mdl.compile(loss = 'mean_squared_error',optimizer='adam')#编译过程 就是建立模型的过程 loss是最优化函数 也是损失函数
        #mean_squared_error函数是指每个输出的输出值减去我们要拟合的标注值 然后平方求和 再开方得到的值让它最小的思路
        #optimizer=sgd表示选择优化器 此时sgd不用加引号 
        #optimizer='adam'选择其他优化器时 要加引号
    
    mdl.fit(X_train,np.array([[0,1] if i ==1 else [1,0] for i in Y_train]),nb_epoch=15000,batch_size=8999)
        #因为输出是one-hot形式 所以这里要转换
        #nb_epoch指一共迭代的次数
        #batch_size指每一次随机梯度下降算法选取的数量
    xy_lst = [(X_train,Y_train),(X_validation,Y_validation),(X_test,Y_test)]
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_
    for i in range(len(xy_lst)):#遍历集合
        jihe_name = ['训练集','验证集','测试集']
        X_part = xy_lst[i][0]
        Y_part = xy_lst[i][1]
        Y_pred = mdl.predict_classes(X_part)#这是需要预测哪个集合 predict_classes这里可以输出分类标注 而不是连续值
        print('神经网络',jihe_name[i])#集合名称
        print('神经网络','-ACC:',accuracy_score(Y_part,Y_pred))#准确率
        print('神经网络',"-REC:",recall_score(Y_part,Y_pred))#召回率
        print('神经网络',"-F1:",f1_score(Y_part,Y_pred))#F值 综合反应
        #反向传播算法可以使用PyBrain
    return
    
    
    models = []#这个models是为了以后调用模型方便
    #储存模型
    models.append(('KNN算法',KNeighborsClassifier(n_neighbors=3)))
    #‘KNN’是元组的第一部分填分类器的名称 后面是分类器的模块
    #因为模型都有fit方法和predict方法
    #所以就不需要这个knn_clf = KNeighborsClassifier(n_neighbors=3)
    #knn_clf.fit(X_train,Y_train)#拟合训练集 这里因为有model 所以先不考虑这部分
    models.append(('高斯朴素贝叶斯',GaussianNB()))
    models.append(('伯努利朴素贝叶斯',BernoulliNB()))
    models.append(('决策树(使用Gini系数)',DecisionTreeClassifier()))#后面可以填入min_impurity_split = 0.1来设置最小不纯度的切分
    models.append(('决策树(使用信息增益)',DecisionTreeClassifier(criterion = 'entropy')))
    models.append(('支持向量机SVM',SVC(C = 1000)))#C是用来控制精度的 越大越好 但计算速度也会慢
    models.append(('随机森林',RandomForestClassifier(n_estimators=81,max_features = None)))#max_features = None指全集
    #bootstrap=False表示没有放回
    #n_estimators=81表示用81个决策树
    models.append(('AdaBoost',AdaBoostClassifier(n_estimators=100)))
    #n_estimators=1000是用来提高弱学习分类器的数量 base_estimator = SVC()和algorithm='SAMME'配合使用
    models.append(('逻辑斯特回归',LogisticRegression(C = 1000,tol = 1e-10,solver = 'sag',max_iter = 10000)))
    #C越大 模型对正则化的影响越小 tol是调精度的 越小精度越高
    #max_iter指最大迭代次数 逻辑斯特回归本质上是线性的 大多数把它当分类器使用
    models.append(('DBDT提升树',GradientBoostingClassifier(max_depth = 6, n_estimators = 100)))
    #max_depth指深度 n_estimators指树的数量

   
    for clf_name,clf in models:#遍历 clf是classifier是分类器的意思
        clf.fit(X_train,Y_train)#拟合训练集
        xy_lst = [(X_train,Y_train),(X_validation,Y_validation),(X_test,Y_test)]
        for i in range(len(xy_lst)):#遍历集合
            jihe_name = ['训练集','验证集','测试集']
            X_part = xy_lst[i][0]
            Y_part = xy_lst[i][1]
            Y_pred = clf.predict(X_part)#这是需要预测哪个集合
            print(clf_name,jihe_name[i])#集合名称
            print(clf_name,'-ACC:',accuracy_score(Y_part,Y_pred))#准确率
            print(clf_name,"-REC:",recall_score(Y_part,Y_pred))#召回率
            print(clf_name,"-F1:",f1_score(Y_part,Y_pred))#F值 综合反应
            #dot_data = StringIO()
            #export_graphviz(clf,out_file=dot_data,
            #                feature_names=f_names,
            #                class_names=['没有离职','离职'],
            #               filled=True,
            #                rounded=True,
            #               special_characters=True)#这是用来输出决策树文件的 out_file=None时 dot_data就可以直接输出了
                            #后面的三个True是用来让决策树更好看用的
                            #dot_data = export_graphviz(clf,out_file=None)和dot_data = StringIO()
                                                                            #export_graphviz(clf,out_file=dot_data)是等效的
                    
            #graph = pydotplus.graph_from_dot_data(dot_data.getvalue())#生成决策树图形 getvalue()在使用StringIO()的时候使用
            #graph.write_pdf('dt_tree(未修正).pdf')
    
    #从结果来看 测试集表现都略低于训练集 所以存在微小的过拟合现象 但差距不大 可以忽略

#回归
def regr_test(features,label):
    print('X',features)
    print('Y',label)
    from sklearn.linear_model import LinearRegression,Ridge,Lasso#回归 Ridge是岭回归 Lasso是lasso回归
    regr = LinearRegression()#这里用了一般的回归  可以应用大多数的情况 根据业务的要求来选择
    #regr = Ridge(alpha = 0)#这里用了岭回归 alpha是设置阿尔法的数值 
    #regr = Lasso(alpha = 0.002)#这里用了Lasso回归
    regr.fit(features.values,label.values)
    Y_pred = regr.predict(features.values)#预测 就是它的拟合值
    print('Coef(参数):',regr.coef_)#它的参数 拟合出来以后确定的参数
    from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score#用来衡量一个回归方程的好坏
    print('MSE(平均平方误差):',mean_squared_error(Y_pred,label.values))
    print('MAE(平均绝对误差):',mean_absolute_error(label.values,Y_pred))
    print('R2(拟合度):',r2_score(label.values,Y_pred))

def main():
    features,label = hr_preprocessing()
    regr_test(features[['number_project','average_monthly_hours']],features['last_evaluation'])
    hr_modeling(features,label)#在用回归时 这个暂时隐去
if __name__=='__main__':
    main()

X        number_project  average_monthly_hours
0                 0.0               0.285047
1                 0.6               0.775701
2                 1.0               0.822430
3                 0.6               0.593458
4                 0.0               0.294393
...               ...                    ...
14993             0.0               0.257009
14994             0.0               0.299065
14995             0.0               0.219626
14996             0.8               0.859813
14997             0.0               0.289720

[14998 rows x 2 columns]
Y 0        0.265625
1        0.781250
2        0.812500
3        0.796875
4        0.250000
           ...   
14993    0.328125
14994    0.187500
14995    0.265625
14996    0.937500
14997    0.250000
Name: last_evaluation, Length: 14998, dtype: float64
Coef(参数): [0.27259418 0.26915592]
MSE(平均平方误差): 0.05953830678507087
MAE(平均绝对误差): 0.20434435188411923
R2(拟合度): 0.1675359138853515
KNN算法 训练集
KNN算法 -ACC: 0.9749944432096022
KNN算法 -REC

/Users/kefengyinyinyin....../anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


支持向量机SVM 训练集
支持向量机SVM -ACC: 0.9628806401422538
支持向量机SVM -REC: 0.9152700186219739
支持向量机SVM -F1: 0.9217065166432253
支持向量机SVM 验证集
支持向量机SVM -ACC: 0.9553333333333334
支持向量机SVM -REC: 0.9055793991416309
支持向量机SVM -F1: 0.9042857142857142
支持向量机SVM 测试集
支持向量机SVM -ACC: 0.9603333333333334
支持向量机SVM -REC: 0.9156293222683264
支持向量机SVM -F1: 0.9175329175329175
随机森林 训练集
随机森林 -ACC: 1.0
随机森林 -REC: 1.0
随机森林 -F1: 1.0
随机森林 验证集
随机森林 -ACC: 0.9876666666666667
随机森林 -REC: 0.9642346208869814
随机森林 -F1: 0.9732851985559566
随机森林 测试集
随机森林 -ACC: 0.987
随机森林 -REC: 0.9571230982019364
随机森林 -F1: 0.9725931131412509
AdaBoost 训练集
AdaBoost -ACC: 0.957768392976217
AdaBoost -REC: 0.9008379888268156
AdaBoost -F1: 0.9105882352941177
AdaBoost 验证集
AdaBoost -ACC: 0.9583333333333334
AdaBoost -REC: 0.8869814020028612
AdaBoost -F1: 0.9084249084249083
AdaBoost 测试集
AdaBoost -ACC: 0.9636666666666667
AdaBoost -REC: 0.9128630705394191
AdaBoost -F1: 0.9237228831350595
逻辑斯特回归 训练集
逻辑斯特回归 -ACC: 0.7901755945765726
逻辑斯特回归 -REC: 0.34543761638733705
逻辑斯特回